# Reading and Visualizing Time Series Dat
Reading from and writing to S3 object storage is a bit different than regular filesystems.   Here we access public read buckets, requester pays buckets and write to a private bucket for Pangeo users.  We will make much use of `fsspec`, which offers filesystem interfaces to S3 (also HTTPS, FTP and many others) in Python.

In [ ]:
import fsspec

In [ ]:
import xarray as xr

#### Open a NetCDF file from an HTTP server

In [ ]:
infile = fsspec.open('simplecache::https://geoport.usgs.esipfed.org/erddap/files/8544pcs-cal_z3/8544pcs-cal_z3.nc')

In [ ]:
ds = xr.open_dataset(infile.open(), engine='h5netcdf')

In [ ]:
ds

In [ ]:
outfile = fsspec.open('simplecache::s3://esip-qhub/usgs/testing/8544pcs-cal_z3.nc', 
                      mode='wb', s3=dict(profile='esip-qhub'))
with outfile as f:
    ds.to_netcdf(f)

#### Read NetCDF data from a bucket

In [ ]:
ncfile = fsspec.open(f's3://esip-qhub/usgs/testing/8544pcs-cal_z3.nc')
ds = xr.open_dataset(ncfile.open())
ds

#### Read NetCDF data from THREDDS OPeNDAP Service  

In [ ]:
ds = xr.open_dataset('http://geoport.usgs.esipfed.org/thredds/dodsC/silt/usgs/Projects/stellwagen/CF-1.6/BUZZ_BAY/2651-A.cdf')
ds

Visualation interlude: plot a time range of data with hvplot

In [ ]:
import hvplot.xarray

ds['T_20'].sel(time=slice('1982-10-01','1982-10-31')).hvplot(grid=True)

#### Read NetCDF data from ERDDAP's Tabledap Service

In [ ]:
ds = xr.open_dataset('http://erddap.sensors.ioos.us/erddap/tabledap/gov_usgs_cmgp_buzz_bay_265')
ds

#### Read NetCDF data from ERDDAP's griddap Service

In [ ]:
url = 'https://geoport.usgs.esipfed.org/erddap/griddap/adcp_grid_5d6e_e2f9_148d'

ds = xr.open_dataset(url)

In [ ]:
ds

In [ ]:
ds['CS_300']

In [ ]:
ds['CS_300'].sel(time=slice('2009-10-01T12:00:00','2009-10-14T12:00:00')).isel(altitude=[0,1,-1]).hvplot(x='time', grid=True)